# Binary Classification with a Bank Churn Dataset
---

## 1. Import Basic Libraries & Set Data Paths
- 기본 라이브러리 import & 데이터 경로 셋

In [1]:
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [6]:
# 데이터 파일 경로
sample_path = 'sample_submission.csv'
train_path = 'train.csv'
test_path = 'test.csv'
origin_path = 'train.csv'  # origin 데이터셋 경로가 필요함

# 데이터 읽기
sample = pd.read_csv(sample_path)
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
origin = pd.read_csv(origin_path)

In [7]:
sample.head()

,id,Exited
0,165034,0.5
1,165035,0.5
2,165036,0.5
3,165037,0.5
4,165038,0.5


In [8]:
test.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [9]:
train.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [10]:
origin.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


---

## 2. Delete Useless & Duplicated Columns
- 불필요하거나 중복되는 컬럼 삭제

In [11]:
def drop_useless(df):
    columns_to_drop = ['id', 'CustomerId', 'Surname', 'RowNumber']
    for col in columns_to_drop:
        if col in df.columns:
            df.drop([col], axis=1, inplace=True)
    return df

# 불필요한 열 제거
train = drop_useless(train)
test = drop_useless(test)
origin = drop_useless(origin)


# train과 origin 데이터 병합 전 열 정렬
train, origin = train.align(origin, join='inner', axis=1)


# train과 origin 데이터 병합
train = pd.concat([train, origin], ignore_index=True)


# 중복 열 제거
train.drop_duplicates(inplace=True)


print(train)

        CreditScore Geography  Gender   Age  Tenure    Balance  NumOfProducts  \
0               668    France    Male  33.0       3       0.00              2   
1               627    France    Male  33.0       1       0.00              2   
2               678    France    Male  40.0      10       0.00              2   
3               581    France    Male  34.0       2  148882.54              1   
4               716     Spain    Male  33.0       5       0.00              2   
...             ...       ...     ...   ...     ...        ...            ...   
165029          667     Spain  Female  33.0       2       0.00              1   
165030          792    France    Male  35.0       3       0.00              1   
165031          565    France    Male  31.0       5       0.00              1   
165032          554     Spain  Female  30.0       7  161533.00              1   
165033          850    France    Male  31.0       1       0.00              1   

        HasCrCard  IsActive

---

## 3. Missing Values Handling
- 결측치 처리

In [12]:
# 결측치 처리
mode_values_train = train.mode().iloc[0] # 데이터프레임의 각 열에 대한 최빈값을 계산합니다.
train.fillna(mode_values_train, inplace=True) # 데이터프레임의 결측치를 최빈값으로 채웁니다. - 이하 동문


# 결측치(Missing Value)는 데이터셋에서 값이 누락된 경우를 말합니다. 
# 이는 데이터 수집 과정에서 정보가 빠졌거나 손실된 결과로 발생할 수 있습니다. 
# 결측치는 분석 및 모델링 과정에서 문제를 일으킬 수 있으므로, 
# 이를 처리하여 데이터를 완전하게 만드는 것이 중요합니다.

# 결측치 처리 방법:
# 1. 삭제: 결측치가 있는 행이나 열을 제거합니다.
# 2. 대체: 결측치를 다른 값으로 채웁니다. 흔히 사용되는 대체 값은 평균, 중간값, 최빈값 등입니다.
# 3. 예측: 결측치를 예측 모델을 사용하여 채웁니다.
    

mode_values_test = test.mode().iloc[0]
test.fillna(mode_values_test, inplace=True)


print(train)

        CreditScore Geography  Gender   Age  Tenure    Balance  NumOfProducts  \
0               668    France    Male  33.0       3       0.00              2   
1               627    France    Male  33.0       1       0.00              2   
2               678    France    Male  40.0      10       0.00              2   
3               581    France    Male  34.0       2  148882.54              1   
4               716     Spain    Male  33.0       5       0.00              2   
...             ...       ...     ...   ...     ...        ...            ...   
165029          667     Spain  Female  33.0       2       0.00              1   
165030          792    France    Male  35.0       3       0.00              1   
165031          565    France    Male  31.0       5       0.00              1   
165032          554     Spain  Female  30.0       7  161533.00              1   
165033          850    France    Male  31.0       1       0.00              1   

        HasCrCard  IsActive

---

## 4. Encoding String Type to Integer Type
- 문자열 타입을 숫자형 타입으로 인코딩

In [14]:
# 인코딩
# LabelEncoder는 문자열 형태의 범주형 데이터를 숫자로 변환해주는 Scikit-learn의 클래스입니다. 
# 예를 들어, 'Male'과 'Female'을 각각 0과 1로 변환하여 모델이 처리할 수 있게 합니다.
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
train['Gender'] = labelencoder.fit_transform(train['Gender']) # train 데이터프레임의 Gender 열을 숫자로 변환합니다.
test['Gender'] = labelencoder.fit_transform(test['Gender']) # test 데이터프레임의 Gender 열을 숫자로 변환합니다.


# pd.get_dummies를 사용한 원핫 인코딩
# pd.get_dummies 함수는 지정된 열(Geography)을 원핫 인코딩하여 데이터프레임의 범주형 데이터를 이진 벡터로 변환합니다.
# 예를 들어, 'Geography' 열의 값이 'France', 'Spain', 'Germany'인 경우, 
# 각 값에 대해 별도의 열이 생성되고, 값이 해당되는 열에는 1, 나머지에는 0이 들어갑니다.
train = pd.get_dummies(train, columns=['Geography'])
test = pd.get_dummies(test, columns=['Geography'])


# train과 test 데이터프레임의 열이 동일하도록 조정
# train 데이터프레임에는 있지만 test 데이터프레임에는 없는 열을 찾습니다.
missing_cols = set(train.columns) - set(test.columns)

# test 데이터프레임에 누락된 열을 추가하고, 해당 열의 값을 0으로 채웁니다. 
# 이는 train과 test의 열 구조를 동일하게 맞추기 위함입니다.
for c in missing_cols: 
    test[c] = 0
    
# test 데이터프레임의 열 순서를 train 데이터프레임과 동일하게 맞춥니다. 
# 단, Exited 열은 제외합니다. 이를 통해 모델 입력 데이터의 일관성을 유지합니다.
test = test[train.columns.drop('Exited')]

In [16]:
# 출력 확인
print(train.head())
print(test.head())

   CreditScore  Gender   Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          668       1  33.0       3       0.00              2        1.0   
1          627       1  33.0       1       0.00              2        1.0   
2          678       1  40.0      10       0.00              2        1.0   
3          581       1  34.0       2  148882.54              1        1.0   
4          716       1  33.0       5       0.00              2        1.0   

   IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0             0.0        181449.97       0              True   
1             1.0         49503.50       0              True   
2             0.0        184866.69       0              True   
3             1.0         84560.88       0              True   
4             1.0         15068.83       0             False   

   Geography_Germany  Geography_Spain  
0              False            False  
1              False            False  
2              False            

---

## 5. Data Normalization
- 데이터 표준화

In [17]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 수치형 열 식별
num_cols = train.select_dtypes(include=[np.number]).columns.tolist()
num_cols.remove('Exited')
num_cols_test = test.select_dtypes(include=[np.number]).columns.tolist()

# StandardScaler를 통해 수치형 데이터를 표준화
# 이 스케일러는 데이터를 평균이 0이고, 표준 편차가 1이 되도록 변환합니다. 
# 즉, 데이터의 각 특성을 평균 0, 분산 1로 만듭니다.
scaler = StandardScaler()

# train 데이터의 num_cols 열에 대해 스케일러를 적용하여 변환합니다.
train[num_cols] = scaler.fit_transform(train[num_cols])

# test 데이터의 num_cols_test 열에 대해 동일한 스케일러를 적용하여 변환합니다. 
# fit_transform은 학습과 변환을 동시에 수행하지만, 
# transform은 이미 학습된 스케일러를 사용하여 변환만 수행합니다.
test[num_cols_test] = scaler.transform(test[num_cols_test])

In [18]:
# MinMaxScaler 통해 수치형 데이터를 표준화
# 이 스케일러는 데이터를 최소값 0, 최대값 1로 변환합니다. 
# 즉, 각 특성을 [0, 1] 범위 내에 있도록 조정합니다.
scaler = MinMaxScaler() 
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols_test] = scaler.transform(test[num_cols_test])

In [19]:
train.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,0.636,1.0,0.202703,0.3,0.000000,0.333333,1.0,0.0,0.907279,0,True,False,False
1,0.554,1.0,0.202703,0.1,0.000000,0.333333,1.0,1.0,0.247483,0,True,False,False
2,0.656,1.0,0.297297,1.0,0.000000,0.333333,1.0,0.0,0.924364,0,True,False,False
3,0.462,1.0,0.216216,0.2,0.593398,0.000000,1.0,1.0,0.422787,0,True,False,False
4,0.732,1.0,0.202703,0.5,0.000000,0.333333,1.0,1.0,0.075293,0,False,False,True


### Why Use Two Different Scalers?
- StandardScaler transforms data based on the mean and standard deviation, making it advantageous when the data follows a normal distribution. 
- On the other hand, MinMaxScaler scales data to a specific range, usually [0, 1], regardless of the data's distribution. This is beneficial when the data does not follow a normal distribution or when we want to restrict the values within a certain range. 
- Using both scalers allows us to compare the impact of each scaling method on the model's performance.

### 왜 두 가지 스케일러를 사용하는가?
- StandardScaler는 평균과 표준편차를 기준으로 변환하므로, 데이터가 정규분포를 따르는 경우 유리합니다.
- MinMaxScaler는 데이터의 분포와 상관없이 일정한 범위 내로 값을 조정하기 때문에, 비정규분포를 따르는 경우나 특정 범위 내에 값을 제한하고자 할 때 유용합니다.
- 두 가지 스케일러를 모두 사용해보는 것은, 각각의 스케일링 방법이 모델의 성능에 미치는 영향을 비교해보기 위함입니다.

---

## 6. Modeling & Evaluation
- 모델링 & 평가

In [21]:
# 이 코드는 LightGBM 모델을 사용하여 이탈 예측 모델을 학습하고 평가하는 과정을 나타냅니다.

# 1. 라이브러리 import
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb


# 2. 데이터 분할
X = train.drop('Exited', axis=1) # 타겟 변수 Exited를 제외한 입력 데이터.
y = train['Exited'] # 타겟 변수 Exited.

# 데이터를 학습 세트와 검증 세트로 분할합니다. 
# test_size=0.2는 데이터를 80:20 비율로 나누며, 
# random_state=42는 결과 재현성을 보장합니다.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# 3. 모델 학습
model = lgb.LGBMClassifier() # LightGBM 분류 모델 객체 생성.
model.fit(X_train, y_train)#학습 데이터를 사용하여 모델 학습.
y_pred_proba = model.predict_proba(X_val)[:, 1] # 검증 데이터에 대한 이탈 확률 예측.

# ROC AUC 점수 계산. 
# ROC AUC는 이진 분류 모델의 성능을 평가하는 지표로, 
# 값이 1에 가까울수록 좋은 성능을 나타냅니다.
roc_auc = roc_auc_score(y_val, y_pred_proba)

[LightGBM] [Info] Number of positive: 27869, number of negative: 104059
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000592 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 859
[LightGBM] [Info] Number of data points in the train set: 131928, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211244 -> initscore=-1.317443
[LightGBM] [Info] Start training from score -1.317443


In [22]:
# ROC AUC 점수 출력.
print(f"ROC AUC Score: {roc_auc}")

ROC AUC Score: 0.8902566306859795


---

## 7. Final Prediction & Submission Save
- 최종 예측 및 결과 저장

In [23]:
# 테스트 데이터에 대한 이탈 확률을 예측합니다.
# predict_proba 함수는 각 클래스에 대한 확률을 반환하며, 
# 여기서는 이탈 클래스(양성 클래스)에 대한 확률을 사용합니다.
y_test_proba = model.predict_proba(test)[:, 1] 
y_test_proba_rounded = np.round(y_test_proba, 1)  

submission = sample.copy()
submission['Exited'] = y_test_proba_rounded
submission.to_csv('submission.csv', index=False)

In [24]:
print(submission)

            id  Exited
0       165034     0.0
1       165035     0.8
2       165036     0.0
3       165037     0.2
4       165038     0.3
...        ...     ...
110018  275052     0.0
110019  275053     0.1
110020  275054     0.0
110021  275055     0.1
110022  275056     0.2

[110023 rows x 2 columns]


---